In [3]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
# transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline
# langchain-huggingface
from langchain_huggingface.llms import HuggingFacePipeline

import torch

# webscrappers
import requests
import wikipedia


import os
import getpass
os.environ["HUGGINGFACEHUB_API_TOKEN"]=getpass.getpass()
repo_id_mistral = "mistralai/Mistral-7B-Instruct-v0.3"


#Text wrapper
import textwrap
def print_text(llm_response):
    wrapper=textwrap.TextWrapper(width=70)
    string=wrapper.fill(text=llm_response)
    return print(string)

2024-08-21 04:27:55.808806: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 04:27:55.809101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 04:27:56.016165: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


 ·····································


In [6]:
# text data
topic = "Oscillating water column"
wiki_article=wikipedia.WikipediaPage(topic).content

## Instantiate model form huggingface
device = "cuda" if torch.cuda.is_available() else "cpu"



llm=HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.3",
                        max_new_tokens=300, 
                        temperature=0.01,
                         model_kwargs={"token":"",
                                       "add_to_git_credential":True},
                        )
# Construct the template for prompt


template= """ Follow these instructions {instructions} to give the answer:
\n\nContext:{context}

Question: {query}

\nAnswer: 
"""

prompt_template=PromptTemplate(input_variables=['query',"context", "instructions"],
                      template=template)





# define the chain with two components: prompt_template, llm

llm_chain= prompt_template | llm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
# Define instructions and context from wikipedia article
instruction="""Instruction: Use your scientific and technical knowldege. Think like a mechanical engineer while giving the answer. If you do not 
know the facts, do not make assumptions and avoid hallucinations and just say your knowldge is limited. Just use the context provided to you and your scientific and technical knowledge."""

## Query from user
user_query="""Pakistan has a huge potential of installing OCWs at its ocean shores?
 If so which type of such plant is feasible for Pakistan considering its geological and environmental concerns?"""

user_query="t"
# format the template based on input variables (context and query) to make a prompt

prompt = prompt_template.format(
    instructions=instruction,
    query=user_query,
    context=wiki_article
)


max_tokens = 512 - len(user_query) - len(instruction)
truncated_context = wiki_article[:max_tokens]
# get response from the model
response = llm_chain.invoke({
    "instructions": instruction,
    "query": user_query,
    "context": wiki_article
})
print_text(response)

 The question does not provide a specific context or question.
However, based on the provided information, it appears that the topic
is about Oscillating Water Columns (OWCs) and their components,
history, major projects, and environmental impact.  OWCs are devices
that harness energy from the oscillation of seawater caused by waves.
They consist of a semi-submerged chamber or hollow, a power take-off
(PTO) system, and a bidirectional turbine. The PTO system converts the
pneumatic power into a desired energy source, such as sound or
electricity. The Wells turbine and Hanna turbine are examples of
bidirectional turbines used in OWCs. The Wells turbine, designed by
Alan Arthur Wells, uses symmetrical airfoils and is most efficient at
low-speed airflows, while the Hanna turbine, invented by John Clark
Hanna, has two rotors with back-to-back asymmetrical airfoils and
offers more torque with a larger operating window.  The history of
OWCs dates back to whistling buoys, which used the air pr

# Vector Store: Chromadb

In [15]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import ArxivLoader

In [31]:
# importing articles from sciencedirect
import urllib.request
import shutil